# Importing Libraries

Following libraries will be used for getting data by web scrapping and api

In [1]:
import requests

from bs4 import BeautifulSoup as soup  # HTML data structure
from urllib.request import urlopen as uReq  # Web client

import mysql.connector as db

import json

# Utility Functions

In [2]:
def forwardGeocoding(country):
    outputFormat = "json"
    parameters = "&pretty=1&limit=1" # Format Api result and limit response to just 1 location (most revelevant)
    key = "key=18d5e5a648744b2e924cd8240138f9d8"
    
    url = "https://api.opencagedata.com/geocode/v1/" + outputFormat + "?" + "q=" + country + parameters + "&" + key    
    
    req = requests.get(url)
    
    json_data = json.loads(req.text)
    
    lat = json_data["results"][0]["geometry"]["lat"]
    lng = json_data["results"][0]["geometry"]["lng"]
    
    return lat, lng
    

In [3]:
def retunErrorGeocoding():
    return 0.0,0.0

# Scrapping Covid Data

We will be scrapping covid19 data from https://en.wikipedia.org/wiki/Template:COVID-19_pandemic_data.

In [4]:
# URl to scrap from.
page_url = "https://en.wikipedia.org/wiki/Template:COVID-19_pandemic_data#covid19-container"

In [5]:
# opens the connection and downloads html page from url
uClient = uReq(page_url)

# parses html into a soup data structure to traverse html
# as if it were a json data type.
page_soup = soup(uClient.read(), "html.parser")
uClient.close()

In [6]:
# Extract required tag from the soup
containers = page_soup.findAll("tr", {"class": ""})

Now extracting required data fields

In [8]:
data = [] # all the data in in a form that can be inserted into databse directely

i = 0
check = 0

flg = True

for container in containers:
    i = i + 1
    if(i > 227): # Container have an additional row that is not of our use
        break
        
    country = container.findAll("th", {"scope": "row"})[1].a.text
    
    dataa = container.findAll("td", {"": ""})
    cases = int(dataa[0].text.replace(',', ''))
    deaths = int(dataa[1].text.replace(',', ''))
    
    recovered = dataa[2].text.replace(',', '')
    recovered = int(recovered) if recovered != 'No data\n' else -1 # Note our data contain "No Data" entries for some countries
    if(recovered == -1):
        check = -1
        
    active = int(cases - deaths - recovered + check)
    check = 0
    
    ####################################################################################################
    # Latitude and Longitude
    
    lat, lng = forwardGeocoding(country) if country != "HNLMS Dolfijn" else retunErrorGeocoding()
    
    #print('{0: <25}'.format(str(country)) + str(lat) + "," + str(lng))
    
    
    ####################################################################################################
    # country varchar(255), latitude float,  longitude float, total int, deaths int, recovered int, active int
    dataPoint = (str(country), float(lat), float(lng),int(cases),int(deaths),int(recovered),int(active))
    data.append(dataPoint)
    
    
    if flg:
        print('\n{0: <7}'.format("Index") + '{0: <25}'.format("Country") + '{0: <15}'.format("Latitude") + '{0: <17}'.format("Longitude") + '{0: <9}'.format("Cases") + '{0: <8}'.format("Deaths") + '{0: <11}'.format("Recovered") + '{0: <10}'.format("Active") + "\n")
        flg = False
    print(" " + '{0: >4}'.format(str(i)) + "  " + '{0: <25}'.format(str(country)) + '{0: <15}'.format(str(lat)) + '{0: <17}'.format(str(lng)) + '{0: <9}'.format(str(cases)) + '{0: <8}'.format(str(deaths)) + '{0: <11}'.format(str(recovered)) + '{0: <10}'.format(str(active)))
    #print(i)


Index  Country                  Latitude       Longitude        Cases    Deaths  Recovered  Active    

    1  United States            39.7837304     -100.4458825     1473415  88237   260146     1125032   
    2  Russia                   64.6863136     97.7453061       272043   2537    63166      206340    
    3  United Kingdom           54.7023545     -3.2765753       240161   34466   -1         205695    
    4  Spain                    39.3262345     -4.8380649       230698   27563   146446     56689     
    5  Italy                    42.6384261     12.674297        223885   31610   120205     72070     
    6  Brazil                   -10.3333333    -53.2            220291   14962   84970      120359    
    7  Germany                  51.0834196     10.4234469       175699   8001    151700     15998     
    8  Turkey                   38.9597594     34.9249653       146457   4055    106133     36269     
    9  France                   46.603354      1.8883335        141919 

   80  Ivory Coast              7.9897371      -5.5679458       2017     24      942        1051      
   81  Sudan                    14.5844444     29.4917691       1964     91      205        1668      
   82  Cuba                     23.0131338     -80.8328748      1840     79      1425       336       
   83  Iceland                  64.9841821     -18.1059013      1802     10      1782       10        
   84  Estonia                  58.7523778     25.3319078       1770     63      934        773       
   85  North Macedonia          41.6171214     21.7168387       1740     97      1251       392       
   86  Guatemala                15.6356088     -89.8988087      1643     30      135        1478      
   87  Lithuania                55.3500003     23.7499997       1534     55      988        491       
   88  Slovakia                 48.7411522     19.4528646       1480     27      1131       322       
   89  Slovenia                 45.8133113     14.4808369       1465     

  160  Brunei                   4.4137155      114.5653908      141      1       134        6         
  161  Mongolia                 46.8250388     103.8499736      135      0       20         115       
  162  Greg Mortimer            51.37339       -1.03495         128      0       -1         128       
  163  Bermuda                  32.3018217     -64.7603583      122      9       66         47        
  164  Cambodia                 13.5066394     104.869423       122      0       122        0         
  165  Mozambique               -19.302233     34.9144977       119      0       42         77        
  166  Guyana                   4.8417097      -58.6416891      116      10      43         63        
  167  Trinidad & Tobago        10.4430243     -61.2613054      116      8       107        1         
  168  Northern Cyprus          35.4594217     33.6783123       108      4       104        0         
  169  Yemen                    16.3471243     47.8915271       106      

In [9]:
data

[('United States', 39.7837304, -100.4458825, 1473415, 88237, 260146, 1125032),
 ('Russia', 64.6863136, 97.7453061, 272043, 2537, 63166, 206340),
 ('United Kingdom', 54.7023545, -3.2765753, 240161, 34466, -1, 205695),
 ('Spain', 39.3262345, -4.8380649, 230698, 27563, 146446, 56689),
 ('Italy', 42.6384261, 12.674297, 223885, 31610, 120205, 72070),
 ('Brazil', -10.3333333, -53.2, 220291, 14962, 84970, 120359),
 ('Germany', 51.0834196, 10.4234469, 175699, 8001, 151700, 15998),
 ('Turkey', 38.9597594, 34.9249653, 146457, 4055, 106133, 36269),
 ('France', 46.603354, 1.8883335, 141919, 27529, 60448, 53942),
 ('Iran', 32.6475314, 54.5643516, 116635, 6902, 91836, 17897),
 ('India', 22.3511148, 78.6677428, 85940, 2752, 30153, 53035),
 ('Peru', -6.8699697, -75.0458515, 84495, 2392, 27147, 54956),
 ('China', 35.000074, 104.999927, 82941, 4633, 78219, 89),
 ('Canada', 61.0666922, -107.9917071, 75007, 5595, 37287, 32125),
 ('Belgium', 50.6402809, 4.6667145, 54989, 9005, 14460, 31524),
 ('Saudi Arabi

# Storing the data in SQL DB

In [ ]:
# Connecting to db
import mysql.connector
conn = mysql.connector.connect( host='Localhost', user='root', passwd='', db='')
cur  = conn.cursor()

Creating our database by the name "BSCS17077_covid", and using it for future insertions

In [ ]:
cur.execute("CREATE Database IF NOT EXISTS BSCS17077_covid;")

cur.execute("USE BSCS17077_covid")

Creating a table by name "covid_details" in db

In [ ]:
cur.execute("DROP TABLE IF EXISTS covid_details")

query = "CREATE TABLE covid_details (country varchar(255), latitude float,  longitude float, total int, deaths int, recovered int, active int);"
cur.execute(query)

Inserting Data in the table

In [ ]:
queryInsert = "INSERT INTO covid_details (country, latitude,  longitude, total, deaths, recovered, active) VALUES (%s, %s, %s, %s, %s, %s, %s)"
cur.executemany(queryInsert,data)

Commiting our data

In [ ]:
conn.commit()

# Queries

Query for countries greater that x active cases

In [ ]:
x = input("Countries with active cases greater than > ")
cur.execute("SELECT * FROM covid_details where active > " + str(x) + " ORDER BY active DESC;")

data = cur.fetchall()

print('\n{0: <7}'.format("Index") + '{0: <25}'.format("Country") + '{0: <15}'.format("Latitude") + '{0: <17}'.format("Longitude") + '{0: <9}'.format("Cases") + '{0: <8}'.format("Deaths") + '{0: <11}'.format("Recovered") + '{0: <10}'.format("Active") + "\n")

i = 1
for dataPoint in data:
    print(" " + '{0: >4}'.format(str(i)) + "  " + '{0: <25}'.format(str(dataPoint[0])) + '{0: <15}'.format(str(dataPoint[1])) + '{0: <17}'.format(str(dataPoint[2])) + '{0: <9}'.format(str(dataPoint[3])) + '{0: <8}'.format(str(dataPoint[4])) + '{0: <11}'.format(str(dataPoint[5])) + '{0: <10}'.format(str(dataPoint[6])))
    i = i + 1


Query for countries greater that x death cases

In [ ]:
x = input("Countries with death cases greater than > ")
cur.execute("SELECT * FROM covid_details where deaths > " + str(x) + " ORDER BY deaths DESC;")

data = cur.fetchall()

print('\n{0: <7}'.format("Index") + '{0: <25}'.format("Country") + '{0: <15}'.format("Latitude") + '{0: <17}'.format("Longitude") + '{0: <9}'.format("Cases") + '{0: <8}'.format("Deaths") + '{0: <11}'.format("Recovered") + '{0: <10}'.format("Active") + "\n")

i = 1
for dataPoint in data:
    print(" " + '{0: >4}'.format(str(i)) + "  " + '{0: <25}'.format(str(dataPoint[0])) + '{0: <15}'.format(str(dataPoint[1])) + '{0: <17}'.format(str(dataPoint[2])) + '{0: <9}'.format(str(dataPoint[3])) + '{0: <8}'.format(str(dataPoint[4])) + '{0: <11}'.format(str(dataPoint[5])) + '{0: <10}'.format(str(dataPoint[6])))
    i = i + 1


Query for countries greater that x recovered cases

In [ ]:
x = input("Countries with recovered cases greater than > ")
cur.execute("SELECT * FROM covid_details where recovered > " + str(x) + " ORDER BY recovered DESC;")

data = cur.fetchall()

print('\n{0: <7}'.format("Index") + '{0: <25}'.format("Country") + '{0: <15}'.format("Latitude") + '{0: <17}'.format("Longitude") + '{0: <9}'.format("Cases") + '{0: <8}'.format("Deaths") + '{0: <11}'.format("Recovered") + '{0: <10}'.format("Active") + "\n")

i = 1
for dataPoint in data:
    print(" " + '{0: >4}'.format(str(i)) + "  " + '{0: <25}'.format(str(dataPoint[0])) + '{0: <15}'.format(str(dataPoint[1])) + '{0: <17}'.format(str(dataPoint[2])) + '{0: <9}'.format(str(dataPoint[3])) + '{0: <8}'.format(str(dataPoint[4])) + '{0: <11}'.format(str(dataPoint[5])) + '{0: <10}'.format(str(dataPoint[6])))
    i = i + 1
